#  Predict Future Sales

## Import data sets

In [1]:
import numpy as np
import pandas as pd

### import data

df = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
items  = pd.read_csv("items.csv")
cats = pd.read_csv('item_categories.csv')

df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [2]:
a = df.item_cnt_day
df[a>500]
df[(df['item_id'] == 9248) & (df['shop_id'] ==12)].item_cnt_day.mean()

28.366666666666667

In [3]:
print(len(list(set(df.item_id))))
print(len(list(set(df.shop_id))))

21807
60


## Data Preprocessing

### Drop Duplicates

In [4]:
df.drop_duplicates(keep='first', inplace=True)
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


According to the result, 6 duplicates were found and dropped.

### Outlier Detection

In [5]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(12,4))
# plt.xlim(df.item_cnt_day.min(), df.item_cnt_day.max()*1.1)
# sns.boxplot(x=df.item_cnt_day)

# plt.figure(figsize=(12,4))
# plt.xlim(df.item_price.min(), df.item_price.max()*1.1)
# sns.boxplot(x=df.item_price)

We see that there are a few obvious outliers, so let's remove them

In [6]:
# df = df[df.item_price<100000]
# df = df[df.item_cnt_day<900]

In [7]:
### Detect outliers with z-score and remove them

def detect_outliers(outliers, data):
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    outliers_index = df.loc[(data < (-3 * std + mean)) | (data > (3 * std + mean))].index.tolist()
    for i in data:
        z_score = (i - mean) / std
        if np.abs(z_score) > threshold:
            outliers.append(i)
    return outliers, outliers_index


train_sales_outliers = []
train_price_outliers = []
train_sales = df.item_cnt_day
train_price = df.item_price

train_sales_outliers, outliers_index = detect_outliers(train_sales_outliers, train_sales)
df = df.drop(outliers_index, axis=0)

train_price_outliers, outliers_index = detect_outliers(train_price_outliers, train_price)
df = df.drop(outliers_index, axis=0)

result = "The training sales outliers are {}".format(np.array(train_sales_outliers))
print(result)
result = "The training prices outliers are {}".format(np.array(train_price_outliers))
print(result)
result = "The size of the cleaned training data is {}".format(np.array(df).size)
print(result)

The training sales outliers are [13. 13. 10. ... 15. 10. 35.]
The training prices outliers are [ 8490.  6190. 13499. ...  7990.  7990.  9990.]
The size of the cleaned training data is 17372358


Locate and remove the items with the price lower than zero, and fill with median value.

In [8]:
train_price = df.item_price
df.loc[train_price < 0, 'item_price']

484683   -1.0
Name: item_price, dtype: float64

One item was detected to be negative, we change the negative price value with mean value of that have same shop id, item_id and date_block_num

In [9]:
print(df.loc[484680:484685,:])
shop_id = int(df.loc[train_price < 0, 'shop_id'])
item_id = int(df.loc[train_price < 0, 'item_id'])
date_block_num = int(df.loc[train_price < 0, 'date_block_num'])
result = "One negative price detected: Shop_id: {}, item_id: {}, date_block_num: {}".format(shop_id, item_id,
                                                                                            date_block_num)
print(result)
median = df[(df.shop_id == shop_id) & (df.item_id == item_id) & (df.date_block_num == date_block_num) & (
        df.item_price > 0)].item_price.median()
df.loc[train_price < 0, 'item_price'] = median

              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
484680  18.05.2013               4       32     3046       799.0           1.0
484681  07.05.2013               4       32     3017       299.0           1.0
484682  23.05.2013               4       32     2973      1249.0           1.0
484683  15.05.2013               4       32     2973        -1.0           1.0
484684  06.05.2013               4       32     2973      2499.0           1.0
484685  02.05.2013               4       32     2972       599.0           1.0
One negative price detected: Shop_id: 32, item_id: 2973, date_block_num: 4


### Substitue 'date' column with 'month' column in the train data set

We are predicting sales based on month, so we need to recalculate the sales for each month.

In [10]:
import datetime
def DateMonth(time):
    day,month,year = time.split('.')
    return int(month)
df['month']=list(map(DateMonth,df['date']))

In [11]:
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,02.01.2013,0,59,22154,999.00,1.0,1
1,03.01.2013,0,25,2552,899.00,1.0,1
2,05.01.2013,0,25,2552,899.00,-1.0,1
3,06.01.2013,0,25,2554,1709.05,1.0,1
4,15.01.2013,0,25,2555,1099.00,1.0,1
...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,10
2935845,09.10.2015,33,25,7460,299.00,1.0,10
2935846,14.10.2015,33,25,7459,349.00,1.0,10
2935847,22.10.2015,33,25,7440,299.00,1.0,10


### Add up day sales into month sales

In [12]:
df = df.drop('date',axis=1).groupby(["item_id","shop_id","date_block_num"]).agg({'item_price':np.mean,'month':'mean','item_cnt_day':'sum'}).reset_index()
df.rename(columns = {"item_cnt_day":"item_cnt_month"},inplace=True)
df

,item_id,shop_id,date_block_num,item_price,month,item_cnt_month
0,0,54,20,58.0,9,1.0
1,1,55,15,4490.0,4,2.0
2,1,55,18,4490.0,7,1.0
3,1,55,19,4490.0,8,1.0
4,1,55,20,4490.0,9,1.0
...,...,...,...,...,...,...
1594190,22168,12,8,799.0,9,1.0
1594191,22168,16,1,799.0,2,1.0
1594192,22168,42,1,799.0,2,1.0
1594193,22168,43,2,798.5,3,1.0


### Add time series columns

As discussed in the LSTM part, we see consider seasonality can improve the model's accuracy. In XGBoost model, let's add time series columns (mark each month as 0-33) instead of adding seasonality, because this might train the model better.

In [13]:
table = df.pivot_table(index=['shop_id', 'item_id'], columns='date_block_num', values='item_cnt_month',aggfunc='sum').fillna(0.0).reset_index()
table['shop_id']= table.shop_id
table['item_id']= table.item_id
df_price = df[['shop_id', 'item_id', 'item_price']].groupby(['item_id','shop_id']).mean().reset_index()
df = pd.merge(table, df_price, on=['shop_id', 'item_id'], how='inner')

In [14]:
df

,shop_id,item_id,0,1,2,3,4,5,6,7,...,25,26,27,28,29,30,31,32,33,item_price
0,0,30,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,265.0
1,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,434.0
2,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0
3,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,347.0
4,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419154,59,22154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0
419155,59,22155,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0
419156,59,22162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,369.0
419157,59,22164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,724.0


### Add item category into train data set

In [15]:
df_category = pd.merge(items,cats,on=['item_category_id'])
df_category

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Кино - DVD
...,...,...,...,...
22165,Элемент питания GP 15AU (LR6) - Блистер 2 шт U...,22095,83,Элементы питания
22166,Элемент питания GP 24A (LR03) - Блистер 2 шт,22096,83,Элементы питания
22167,Элемент питания GP 24AU (LR03) - Блистер 2 шт ...,22097,83,Элементы питания
22168,Элемент питания КОСМОС LR03 2*BL,22098,83,Элементы питания


In [16]:
df_category['item_category_id'].astype(np.int32)
df_category['new_item_category'] = 'other'
df_category['new_item_category'][df_category['item_category_id'].between(1,8,inclusive=True)] = 'Digital Appliances'
df_category['new_item_category'][df_category['item_category_id'].between(10,18,inclusive=True)] = 'Consoles'
df_category['new_item_category'][df_category['item_category_id'].between(18,25,inclusive=True)] = 'Consoles Games'
df_category['new_item_category'][df_category['item_category_id'].between(26,27,inclusive=True)] = 'Phone games'
df_category['new_item_category'][df_category['item_category_id'].between(28,31,inclusive=True)] = 'CD games'
df_category['new_item_category'][df_category['item_category_id'].between(32,36,inclusive=True)] = 'Card'
df_category['new_item_category'][df_category['item_category_id'].between(37,42,inclusive=True)] = 'Movie'
df_category['new_item_category'][df_category['item_category_id'].between(43,54,inclusive=True)] = 'Books'
df_category['new_item_category'][df_category['item_category_id'].between(55,60,inclusive=True)] = 'Music'
df_category['new_item_category'][df_category['item_category_id'].between(61,72,inclusive=True)] = 'Gifts'
df_category['new_item_category'][df_category['item_category_id'].between(73,79,inclusive=True)] = 'Soft'

C:\Users\YOOJI\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\YOOJI\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\YOOJI\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [17]:
df = pd.merge(df,df_category[['item_id','new_item_category']],on=['item_id'],how='left')

In [18]:
df

,shop_id,item_id,0,1,2,3,4,5,6,7,...,26,27,28,29,30,31,32,33,item_price,new_item_category
0,0,30,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,265.0,Movie
1,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,434.0,Movie
2,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0,Movie
3,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,347.0,Movie
4,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,247.0,Movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419154,59,22154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0,Movie
419155,59,22155,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0,Movie
419156,59,22162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,369.0,Movie
419157,59,22164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,724.0,Movie


In [19]:
df_cate = pd.get_dummies(df['new_item_category'],drop_first=True)
df = pd.concat([df,df_cate],axis=1)
df.drop(['new_item_category'], axis=1, inplace=True)
df

,shop_id,item_id,0,1,2,3,4,5,6,7,...,Card,Consoles,Consoles Games,Digital Appliances,Gifts,Movie,Music,Phone games,Soft,other
0,0,30,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
1,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
3,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419154,59,22154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419155,59,22155,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419156,59,22162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419157,59,22164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


### Standard Scaler

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col = ['item_price']
df[col] = scaler.fit_transform(df[col])

In [21]:
df

,shop_id,item_id,0,1,2,3,4,5,6,7,...,Card,Consoles,Consoles Games,Digital Appliances,Gifts,Movie,Music,Phone games,Soft,other
0,0,30,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
1,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
3,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419154,59,22154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419155,59,22155,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419156,59,22162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
419157,59,22164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


## Apply XGBoost model

In [22]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [23]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

In [24]:
xgb = XGBRegressor()
parameters = {
              'learning_rate': [0.5, 0.7], 
              'max_depth': [10, 15],
              'subsample': [0.7],
              'n_estimators': [100, 200],
              'eval_metric': ['rmse']}
xgb_grid = GridSearchCV(xgb, parameters, cv=3, n_jobs=-1, verbose=True)


X = df.drop(33, axis=1)
Y = df[33]

xgb_grid.fit(X, Y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 16.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             iid='warn', n_jobs=-

In [25]:
#Here are our optimal hyperparameters
best_grid = xgb_grid.best_estimator_
best_grid

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.5, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
regressor = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                            colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
                            gamma=0, gpu_id=-1, importance_type='gain',
                            interaction_constraints=None, learning_rate=0.5,max_delta_step=0,
                            max_depth=10, min_child_weight=1, missing=np.nan,
                            monotone_constraints=None, n_estimators=100, n_jobs=0,
                            num_parallel_tree=1, objective='reg:squarederror', random_state=0,
                            reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
                            tree_method='exact', validate_parameters=1, verbosity=None)

### Model Validation

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df, df[33], test_size = 0.3, random_state=5)

In [41]:
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [42]:
from sklearn.metrics import mean_squared_error
from numpy import sqrt
rmse = sqrt(mean_squared_error(y_test.tolist(),y_pred))
print('Val RMSE: %.3f' % rmse)

Val RMSE: 0.091


## Final Prediction

In [43]:
#train xgboost model with whole data set
X = df
y = df[33]
print(y)
regressor.fit(X, y)
#predict kaggle test set
import copy
col = list(range(0,33))
df_test = copy.deepcopy(df)
df_test[col] = df[np.add(col, 1)].values
df_test[33] = 0
df_test[33] = regressor.predict(df_test)

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
419154    0.0
419155    0.0
419156    0.0
419157    0.0
419158    0.0
Name: 33, Length: 419159, dtype: float64


In [44]:
test = pd.read_csv('test.csv')

output = pd.merge(test, df_test, on=['shop_id', 'item_id'], how='left').fillna(0)
output = output[['ID', 33]]
output.rename(columns={33: "item_cnt_month"}, inplace=True)
output['item_cnt_month'] = np.clip(output['item_cnt_month'], 0, 20)
output

,ID,item_cnt_month
0,0,5.960464e-08
1,1,0.000000e+00
2,2,5.960464e-08
3,3,5.960464e-08
4,4,0.000000e+00
...,...,...
214195,214195,5.960464e-08
214196,214196,0.000000e+00
214197,214197,5.960464e-08
214198,214198,0.000000e+00


In [45]:
output.to_csv(r'D:\Documents\529HWS\submission.csv', index=False)

Final submission of Kaggle score is 1.24480